In [1]:
pip install --pre deepchem

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install dgllife

In [3]:
import deepchem as dc
import pandas as pd
import numpy as np
from deepchem.models import GCNModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\MAIYP\anaconda3\envs\crosswalk\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\MAIYP\anaconda3\envs\crosswalk\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loadin

In [8]:
#import iris dataset
df = pd.read_csv('data/iris.csv', header=None)
df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
species_map = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
df['species'] = df['species'].map(species_map)

#standardize features
scaler = StandardScaler()
features = df.iloc[:, :-1].values
features_scaled = scaler.fit_transform(features)
labels = df['species'].values



,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [5]:
#split data with indices
indices = np.arange(len(features_scaled))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)

#split features and labels
train_features = features_scaled[train_idx]
test_features = features_scaled[test_idx]
train_labels = labels[train_idx]
test_labels = labels[test_idx]

# Cell 4: Create graph structures
def create_graph(features):
    n_samples = len(features)
    k = 5  #nb neighbors
    edge_index = []
    
    for i in range(n_samples):
        distances = []
        for j in range(n_samples):
            if i != j:
                dist = np.linalg.norm(features[i] - features[j])
                distances.append((j, dist))
        
        nearest_neighbors = sorted(distances, key=lambda x: x[1])[:k]
        for neighbor, _ in nearest_neighbors:
            edge_index.append([i, neighbor])
            edge_index.append([neighbor, i])
    
    edge_index = np.array(edge_index, dtype=np.int32).T
    return dc.feat.graph_data.GraphData(
        node_features=features.astype(np.float32),
        edge_index=edge_index,
        num_nodes=n_samples,
        num_edges=len(edge_index)
    )

#separation train and test
train_graph = create_graph(train_features)
test_graph = create_graph(test_features)

In [6]:
#create deepchem datasets train/test
train_dataset = dc.data.NumpyDataset([train_graph], train_labels.reshape(-1, 1))
test_dataset = dc.data.NumpyDataset([test_graph], test_labels.reshape(-1, 1))

n_features = features_scaled.shape[1] 

In [15]:
model = GCNModel(mode='classification', n_tasks=1,
                 batch_size=16, learning_rate=0.001, number_atom_features=n_features)

#train model
model.fit(train_dataset, nb_epoch=10)


DGLError: [16:01:39] C:\Users\Administrator\dgl-0.5\src\runtime\c_runtime_api.cc:88: Check failed: allow_missing: Device API gpu is not enabled. Please install the cuda version of dgl.